In [1]:
import tensorflow as tf
import os

os.chdir('..')

# model_path = r'artifacts\prepare_base_model\base_model_updated.h5'

# m = tf.keras.models.load_model(model_path)

In [2]:
os.getcwd()

'd:\\Full Stack Data Science\\Python Project\\DL\\Current Batch\\Deep_CNN_Classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path
from box import ConfigBox

@dataclass(frozen=True)
class ModelEvaluationConfig:
    trained_model_filepath: Path
    test_data_dir: Path
    params: ConfigBox
    scores_filepath: Path
    remote_server_uri: str
    train_params: dict

In [5]:
from DeepClassifier.constants import *
from DeepClassifier.utils import *

class ConfigurationManager:

    def __init__(
        self,
        config_filepath: Path=CONFIG_FILEPATH,
        params_filepath: Path=PARAMS_FILEPATH,
    ):
        self.config = read_yaml_file(config_filepath)
        make_directories([self.config.root_artifact_dir])
        self.params = read_yaml_file(params_filepath)

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        model_evaluation_config_info = self.config.model_evaluation

        model_evaluation_config = ModelEvaluationConfig(
            trained_model_filepath=model_evaluation_config_info.trained_model_filepath,
            test_data_dir=model_evaluation_config_info.test_data_dir,
            scores_filepath=model_evaluation_config_info.scores_filepath,
            remote_server_uri=model_evaluation_config_info.remote_server_uri,
            params=self.params,
            train_params=dict(model_evaluation_config_info.train_params)
        )
        return model_evaluation_config

In [6]:
c = ConfigurationManager()
s = c.get_model_evaluation_config()
print(s)

ModelEvaluationConfig(trained_model_filepath='artifacts/model_training/model.h5', test_data_dir='artifacts/data_ingestion', params=ConfigBox({'AUGMENTATION': True, 'IMAGE_SHAPE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}), scores_filepath='scores.json', remote_server_uri='https://dagshub.com/TewariUtkarsh/Deep_CNN_Classifier.mlflow', train_params={'epochs': 1})


In [14]:
# ModelEvaluation(s)

In [9]:
import tensorflow as tf
from pathlib import Path
from DeepClassifier.utils import save_json
from DeepClassifier.entity import ModelEvaluationConfig
import mlflow
import mlflow.tensorflow
from urllib.parse import urlparse


class ModelEvaluation:
    def __init__(
            self,
            model_evaluation_config:ModelEvaluationConfig,

        ):
        self.model_evaluation_config = model_evaluation_config

    def _get_valid_generator(self):
        data_generator_kwargs = dict(
            rescale=1/255.,
            validation_split=0.30,
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        data_flow_kwargs = dict(
            target_size=self.model_evaluation_config.params.IMAGE_SHAPE[:-1],
            batch_size=self.model_evaluation_config.params.BATCH_SIZE,
            interpolation='bilinear'
        )


        self.valid_data_generator =valid_generator.flow_from_directory(
            self.model_evaluation_config.test_data_dir,
            subset='validation',
            shuffle=False,
            **data_flow_kwargs
        )

        
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluate(self):
        self._get_valid_generator()
        self.model = self.load_model(path=Path(self.model_evaluation_config.trained_model_filepath))
        self.scores = self.model.evaluate(self.valid_data_generator)
        self.save_scores()

    def save_scores(self):
        self.scores = {"loss": self.scores[0], "accuracy": self.scores[1]}
        save_json(path=Path(self.model_evaluation_config.scores_filepath),data=self.scores)
        
    def log_into_mlflow(self):

        os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/TewariUtkarsh/Deep_CNN_Classifier.mlflow'
        os.environ['MLFLOW_TRACKING_USERNAME'] = 'TewariUtkarsh'
        os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd7f92dada13480fbb4d1cabdef4818878fa0fc28'
        
        mlflow.set_tracking_uri(self.model_evaluation_config.remote_server_uri)
        
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run(): 
            mlflow.log_metrics(
                self.scores
            )

            mlflow.log_params(
                self.model_evaluation_config.train_params
            )

            
            if tracking_uri_type_store != 'file':
                mlflow.keras.log_model(
                    self.model,
                    'saved_model',
                    registered_model_name='VGG16Model'
                )
            else:
                mlflow.keras.log_model(
                    self.model,
                    'save_model'
                )

        

In [10]:
try:
    config = ConfigurationManager()
    # model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(s)
    model_evaluation.evaluate()
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 7499 images belonging to 1 classes.
469/469 [==============================] - 116s 246ms/step - loss: 17.4278 - accuracy: 0.4618
INFO:tensorflow:Assets written to: C:\Users\DJ\AppData\Local\Temp\tmplurkp_ba\model\data\model\assets


Successfully registered model ''.
2022/10/21 18:03:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
